In [ ]:
import shamirs
import numpy as np
import random
import time
import matplotlib.pyplot as plt

# Function to split the secret into smaller chunks
def split_secret(secret_int, chunk_size):
    binary_str = bin(secret_int)[2:].zfill(chunk_size * ((len(bin(secret_int)[2:]) + chunk_size - 1) // chunk_size))
    chunks = [int(binary_str[i:i + chunk_size], 2) for i in range(0, len(binary_str), chunk_size)]
    return chunks

# Function to combine shares for each chunk into a single share set
def combine_shares(chunk_shares):
    combined_shares = []
    for i in range(len(chunk_shares[0])):
        combined_shares.append(tuple(share[i] for share in chunk_shares))
    return combined_shares

# Function to apply Shamir's Secret Sharing to each chunk
def shamir_split(secret_int, chunk_size, quantity, threshold):
    chunks = split_secret(secret_int, chunk_size)
    shares = [shamirs.shares(chunk, quantity=quantity, threshold=threshold) for chunk in chunks]
    combined_shares = combine_shares(shares)
    return combined_shares

# Function to recover the original secret from shares
def shamir_combine(shares, chunk_size):
    chunks = [shamirs.interpolate(share_set) for share_set in zip(*shares)]
    binary_str = ''.join(bin(chunk)[2:].zfill(chunk_size) for chunk in chunks)
    return int(binary_str, 2)

# Testing with the same loop as before
iterations = 1

# Arrays to store runtimes
encoder_runtimes_shamir = []
decoder_runtimes_shamir = []
encoder_runtimes_xor = []
decoder_runtimes_E1_E2 = []
decoder_runtimes_E1_E3 = []
decoder_runtimes_E2_E3 = []

#secret = b"This is the cryptography protocol"
with open('input.txt', 'rb') as f:
    secret = f.read().strip()

n = 3
k = 2

# Convert the secret to an integer
secret_int = int.from_bytes(secret, byteorder='big')

# Choose a chunk size that is smaller than the modulus (e.g., 32 bits)
chunk_size = 32

for _ in range(iterations):
    # Shamir's Scheme
    start_time = time.time()
    shamir_shares = shamir_split(secret_int, chunk_size, quantity=n, threshold=k)
    encoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    recovered_secret_int = shamir_combine(shamir_shares, chunk_size)
    decoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    # XOR-based Secret Sharing using bytes
    start_time = time.time()
    E1, E2, E3 = encode(secret_int)
    encoder_runtimes_xor.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E1, E2, E1, E2, E3)
    decoder_runtimes_E1_E2.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E1, E3, E1, E2, E3)
    decoder_runtimes_E1_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    start_time = time.time()
    decode(E2, E3, E1, E2, E3)
    decoder_runtimes_E2_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

# Calculating the averages
avg_encoder_runtime_shamir = np.mean(encoder_runtimes_shamir)
print(avg_encoder_runtime_shamir)
avg_decoder_runtime_shamir = np.mean(decoder_runtimes_shamir)
print(avg_decoder_runtime_shamir)
avg_encoder_runtime_xor = np.mean(encoder_runtimes_xor)
print(avg_encoder_runtime_xor)
avg_decoder_runtime_E1_E2 = np.mean(decoder_runtimes_E1_E2)
print(avg_decoder_runtime_E1_E2)
avg_decoder_runtime_E1_E3 = np.mean(decoder_runtimes_E1_E3)
print(avg_decoder_runtime_E1_E3)
avg_decoder_runtime_E2_E3 = np.mean(decoder_runtimes_E2_E3)
print(avg_decoder_runtime_E2_E3)

# Calculate speedups
encoder_speedup = avg_encoder_runtime_shamir / avg_encoder_runtime_xor
decoder_speedup_E1_E2 = avg_decoder_runtime_shamir / avg_decoder_runtime_E1_E2
decoder_speedup_E1_E3 = avg_decoder_runtime_shamir / avg_decoder_runtime_E1_E3
decoder_speedup_E2_E3 = avg_decoder_runtime_shamir / avg_decoder_runtime_E2_E3

print(f"Encoder speedup (XOR vs Shamir): {encoder_speedup:.2f}x")
print(f"Decoder speedup (E1,E2 vs Shamir): {decoder_speedup_E1_E2:.2f}x")
print(f"Decoder speedup (E1,E3 vs Shamir): {decoder_speedup_E1_E3:.2f}x")
print(f"Decoder speedup (E2,E3 vs Shamir): {decoder_speedup_E2_E3:.2f}x")

# Plotting the results
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 1)
plt.bar(['Shamir', 'XOR'], [avg_encoder_runtime_shamir, avg_encoder_runtime_xor], color=['#1f77b4', '#ff7f0e'])
plt.title('Average Encoding Runtime (2, 3)', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.subplot(1, 2, 2)
plt.bar(['Shamir', 'E1,E2', 'E1,E3', 'E2,E3'],
        [avg_decoder_runtime_shamir, avg_decoder_runtime_E1_E2, avg_decoder_runtime_E1_E3, avg_decoder_runtime_E2_E3],
        color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Average Decoding Runtime (2, 3)', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.tight_layout()
plt.show()

# Plotting the speedup
plt.figure(figsize=(10, 7), dpi=100)
plt.bar(['Encoder', 'E1,E2', 'E1,E3', 'E2,E3'],
        [encoder_speedup, decoder_speedup_E1_E2, decoder_speedup_E1_E3, decoder_speedup_E2_E3],
        color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
plt.title('Speedup of XOR-based Scheme Over Shamir\'s Scheme', fontsize=24)
plt.ylabel('Speedup Factor', fontsize=24)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.tight_layout()
plt.show()


In [ ]:
import shamirs
import numpy as np
import random
import time
import matplotlib.pyplot as plt

# Function to split the secret into smaller chunks
def split_secret(secret_int, chunk_size):
    binary_str = bin(secret_int)[2:].zfill(chunk_size * ((len(bin(secret_int)[2:]) + chunk_size - 1) // chunk_size))
    chunks = [int(binary_str[i:i + chunk_size], 2) for i in range(0, len(binary_str), chunk_size)]
    return chunks

# Function to combine shares for each chunk into a single share set
def combine_shares(chunk_shares):
    combined_shares = []
    for i in range(len(chunk_shares[0])):
        combined_shares.append(tuple(share[i] for share in chunk_shares))
    return combined_shares

# Function to apply Shamir's Secret Sharing to each chunk
def shamir_split(secret_int, chunk_size, quantity, threshold):
    chunks = split_secret(secret_int, chunk_size)
    shares = [shamirs.shares(chunk, quantity=quantity, threshold=threshold) for chunk in chunks]
    combined_shares = combine_shares(shares)
    return combined_shares

# Function to recover the original secret from shares
def shamir_combine(shares, chunk_size):
    chunks = [shamirs.interpolate(share_set) for share_set in zip(*shares)]
    binary_str = ''.join(bin(chunk)[2:].zfill(chunk_size) for chunk in chunks)
    return int(binary_str, 2)

# XOR-based Secret Sharing using bytes
def encode(secret):
    M = bin(secret)[2:]
    if len(M) % 2 != 0:
        M += '0'
    half_len = len(M) // 2
    M1 = bin_str_to_bytes(M[:half_len])
    M2 = bin_str_to_bytes(M[half_len:])
    
    R1 = random.randbytes(len(M1))
    R2 = random.randbytes(len(M2))
    
    E1 = R1 + bytes(a ^ b for a, b in zip(M2, R2))
    E2 = bytes(a ^ b for a, b in zip(M1, R1)) + R2
    E3 = bytes(a ^ b for a, b in zip(M1, R2)) + bytes(a ^ b for a, b in zip(M2, R1))
    
    return E1, E2, E3

def decode(part1, part2, E1, E2, E3):
    half_len = len(part1) // 2

    if part1 == E1 and part2 == E2:
        R1 = part1[:half_len]
        R2 = part2[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R1))
        M2 = bytes(a ^ b for a, b in zip(part1[half_len:], R2))
    elif part1 == E1 and part2 == E3:
        R1 = part1[:half_len]
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
        R2 = bytes(a ^ b for a, b in zip(part1[half_len:], M2))
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
    elif part1 == E2 and part2 == E3:
        R2 = part1[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
        R1 = bytes(a ^ b for a, b in zip(part1[:half_len], M1))
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
    else:
        raise ValueError("Invalid parts provided for decoding.")

    M = M1 + M2
    return int.from_bytes(M, byteorder='big')

# List of input files
input_files = [
    'input_1_bytes.txt',
    'input_10_bytes.txt',
    'input_100_bytes.txt',
    'input_1000_bytes.txt',
    'input_10000_bytes.txt'
]

# Arrays to store average runtimes across files
avg_encoder_runtimes_shamir = []
avg_decoder_runtimes_shamir = []
avg_encoder_runtimes_xor = []
avg_decoder_runtimes_E1_E2 = []
avg_decoder_runtimes_E1_E3 = []
avg_decoder_runtimes_E2_E3 = []

n = 3  # Number of shares
k = 2  # Threshold

# Choose a chunk size that is smaller than the modulus (e.g., 32 bits)
chunk_size = 32

for input_file in input_files:
    # Read the secret from the file
    with open(input_file, 'rb') as f:
        secret = f.read().strip()

    # Convert the secret to an integer
    secret_int = int.from_bytes(secret, byteorder='big')

    # Arrays to store runtimes for this file
    encoder_runtimes_shamir = []
    decoder_runtimes_shamir = []
    encoder_runtimes_xor = []
    decoder_runtimes_E1_E2 = []
    decoder_runtimes_E1_E3 = []
    decoder_runtimes_E2_E3 = []

    # Perform the encoding and decoding multiple times (iterations) for averaging
    iterations = 10
    for _ in range(iterations):
        # Shamir's Scheme
        start_time = time.time()
        shamir_shares = shamir_split(secret_int, chunk_size, quantity=n, threshold=k)
        encoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        recovered_secret_int = shamir_combine(shamir_shares, chunk_size)
        decoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        # XOR-based Secret Sharing using bytes
        start_time = time.time()
        E1, E2, E3 = encode(secret_int)
        encoder_runtimes_xor.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E1, E2, E1, E2, E3)
        decoder_runtimes_E1_E2.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E1, E3, E1, E2, E3)
        decoder_runtimes_E1_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E2, E3, E1, E2, E3)
        decoder_runtimes_E2_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    # Calculating the averages for this file
    avg_encoder_runtimes_shamir.append(np.mean(encoder_runtimes_shamir))
    avg_decoder_runtimes_shamir.append(np.mean(decoder_runtimes_shamir))
    avg_encoder_runtimes_xor.append(np.mean(encoder_runtimes_xor))
    avg_decoder_runtimes_E1_E2.append(np.mean(decoder_runtimes_E1_E2))
    avg_decoder_runtimes_E1_E3.append(np.mean(decoder_runtimes_E1_E3))
    avg_decoder_runtimes_E2_E3.append(np.mean(decoder_runtimes_E2_E3))

# Plotting the results as line graphs
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 1)
plt.plot(input_files, avg_encoder_runtimes_shamir, label='Shamir Encoder', marker='o', color='#1f77b4')
plt.plot(input_files, avg_encoder_runtimes_xor, label='XOR Encoder', marker='o', color='#ff7f0e')
plt.title('Average Encoding Runtime for Different Inputs', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(input_files, avg_decoder_runtimes_shamir, label='Shamir Decoder', marker='o', color='#1f77b4')
plt.plot(input_files, avg_decoder_runtimes_E1_E2, label='E1,E2 Decoder', marker='o', color='#ff7f0e')
plt.plot(input_files, avg_decoder_runtimes_E1_E3, label='E1,E3 Decoder', marker='o', color='#2ca02c')
plt.plot(input_files, avg_decoder_runtimes_E2_E3, label='E2,E3 Decoder', marker='o', color='#d62728')
plt.title('Average Decoding Runtime for Different Inputs', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import shamirs
import numpy as np
import random
import time
import matplotlib.pyplot as plt

# Function to split the secret into smaller chunks
def split_secret(secret_int, chunk_size):
    binary_str = bin(secret_int)[2:].zfill(chunk_size * ((len(bin(secret_int)[2:]) + chunk_size - 1) // chunk_size))
    chunks = [int(binary_str[i:i + chunk_size], 2) for i in range(0, len(binary_str), chunk_size)]
    return chunks

# Function to combine shares for each chunk into a single share set
def combine_shares(chunk_shares):
    combined_shares = []
    for i in range(len(chunk_shares[0])):
        combined_shares.append(tuple(share[i] for share in chunk_shares))
    return combined_shares

# Function to apply Shamir's Secret Sharing to each chunk
def shamir_split(secret_int, chunk_size, quantity, threshold):
    chunks = split_secret(secret_int, chunk_size)
    shares = [shamirs.shares(chunk, quantity=quantity, threshold=threshold) for chunk in chunks]
    combined_shares = combine_shares(shares)
    return combined_shares

# Function to recover the original secret from shares
def shamir_combine(shares, chunk_size):
    chunks = [shamirs.interpolate(share_set) for share_set in zip(*shares)]
    binary_str = ''.join(bin(chunk)[2:].zfill(chunk_size) for chunk in chunks)
    return int(binary_str, 2)

# XOR-based Secret Sharing using bytes
def encode(secret):
    M = bin(secret)[2:]
    if len(M) % 2 != 0:
        M += '0'
    half_len = len(M) // 2
    M1 = M[:half_len].encode()
    M2 = M[half_len:].encode()
    
    R1 = random.randbytes(len(M1))
    R2 = random.randbytes(len(M2))
    
    E1 = R1 + bytes(a ^ b for a, b in zip(M2, R2))
    E2 = bytes(a ^ b for a, b in zip(M1, R1)) + R2
    E3 = bytes(a ^ b for a, b in zip(M1, R2)) + bytes(a ^ b for a, b in zip(M2, R1))
    
    return E1, E2, E3

def decode(part1, part2, E1, E2, E3):
    half_len = len(part1) // 2

    if part1 == E1 and part2 == E2:
        R1 = part1[:half_len]
        R2 = part2[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R1))
        M2 = bytes(a ^ b for a, b in zip(part1[half_len:], R2))
    elif part1 == E1 and part2 == E3:
        R1 = part1[:half_len]
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
        R2 = bytes(a ^ b for a, b in zip(part1[half_len:], M2))
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
    elif part1 == E2 and part2 == E3:
        R2 = part1[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
        R1 = bytes(a ^ b for a, b in zip(part1[:half_len], M1))
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
    else:
        raise ValueError("Invalid parts provided for decoding.")

    M = M1 + M2
    return int.from_bytes(M, byteorder='big')

# List of input files
input_files = [
    'input_1_bytes.txt',
    'input_10_bytes.txt',
    'input_100_bytes.txt',
    'input_1000_bytes.txt',
    'input_10000_bytes.txt'
]

# Arrays to store average runtimes across files
avg_encoder_runtimes_shamir = []
avg_decoder_runtimes_shamir = []
avg_encoder_runtimes_xor = []
avg_decoder_runtimes_E1_E2 = []
avg_decoder_runtimes_E1_E3 = []
avg_decoder_runtimes_E2_E3 = []

n = 3  # Number of shares
k = 2  # Threshold

# Choose a chunk size that is smaller than the modulus (e.g., 32 bits)
chunk_size = 32

for input_file in input_files:
    # Read the secret from the file
    with open(input_file, 'rb') as f:
        secret = f.read().strip()

    # Convert the secret to an integer
    secret_int = int.from_bytes(secret, byteorder='big')

    # Arrays to store runtimes for this file
    encoder_runtimes_shamir = []
    decoder_runtimes_shamir = []
    encoder_runtimes_xor = []
    decoder_runtimes_E1_E2 = []
    decoder_runtimes_E1_E3 = []
    decoder_runtimes_E2_E3 = []

    # Perform the encoding and decoding multiple times (iterations) for averaging
    iterations = 10
    for _ in range(iterations):
        # Shamir's Scheme
        start_time = time.time()
        shamir_shares = shamir_split(secret_int, chunk_size, quantity=n, threshold=k)
        encoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        recovered_secret_int = shamir_combine(shamir_shares, chunk_size)
        decoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        # XOR-based Secret Sharing using bytes
        start_time = time.time()
        E1, E2, E3 = encode(secret_int)
        encoder_runtimes_xor.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E1, E2, E1, E2, E3)
        decoder_runtimes_E1_E2.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E1, E3, E1, E2, E3)
        decoder_runtimes_E1_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E2, E3, E1, E2, E3)
        decoder_runtimes_E2_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    # Calculating the averages for this file
    avg_encoder_runtimes_shamir.append(np.mean(encoder_runtimes_shamir))
    avg_decoder_runtimes_shamir.append(np.mean(decoder_runtimes_shamir))
    avg_encoder_runtimes_xor.append(np.mean(encoder_runtimes_xor))
    avg_decoder_runtimes_E1_E2.append(np.mean(decoder_runtimes_E1_E2))
    avg_decoder_runtimes_E1_E3.append(np.mean(decoder_runtimes_E1_E3))
    avg_decoder_runtimes_E2_E3.append(np.mean(decoder_runtimes_E2_E3))

# Print the calculated data
print("Average Encoder Runtimes (Shamir):", avg_encoder_runtimes_shamir)
print("Average Decoder Runtimes (Shamir):", avg_decoder_runtimes_shamir)
print("Average Encoder Runtimes (XOR):", avg_encoder_runtimes_xor)
print("Average Decoder Runtimes (E1,E2):", avg_decoder_runtimes_E1_E2)
print("Average Decoder Runtimes (E1,E3):", avg_decoder_runtimes_E1_E3)
print("Average Decoder Runtimes (E2,E3):", avg_decoder_runtimes_E2_E3)

# Plotting the results as line graphs
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 1)
plt.plot(input_files, avg_encoder_runtimes_shamir, label='Shamir Encoder', marker='o', color='#1f77b4')
plt.plot(input_files, avg_encoder_runtimes_xor, label='XOR Encoder', marker='o', color='#ff7f0e')
plt.title('Average Encoding Runtime for Different Inputs', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(input_files, avg_decoder_runtimes_shamir, label='Shamir Decoder', marker='o', color='#1f77b4')
plt.plot(input_files, avg_decoder_runtimes_E1_E2, label='E1,E2 Decoder', marker='o', color='#ff7f0e')
plt.plot(input_files, avg_decoder_runtimes_E1_E3, label='E1,E3 Decoder', marker='o', color='#2ca02c')
plt.plot(input_files, avg_decoder_runtimes_E2_E3, label='E2,E3 Decoder', marker='o', color='#d62728')
plt.title('Average Decoding Runtime for Different Inputs', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import shamirs
import numpy as np
import random
import time
import matplotlib.pyplot as plt

# Function to split the secret into smaller chunks
def split_secret(secret_int, chunk_size):
    binary_str = bin(secret_int)[2:].zfill(chunk_size * ((len(bin(secret_int)[2:]) + chunk_size - 1) // chunk_size))
    chunks = [int(binary_str[i:i + chunk_size], 2) for i in range(0, len(binary_str), chunk_size)]
    return chunks

# Function to combine shares for each chunk into a single share set
def combine_shares(chunk_shares):
    combined_shares = []
    for i in range(len(chunk_shares[0])):
        combined_shares.append(tuple(share[i] for share in chunk_shares))
    return combined_shares

# Function to apply Shamir's Secret Sharing to each chunk
def shamir_split(secret_int, chunk_size, quantity, threshold):
    chunks = split_secret(secret_int, chunk_size)
    shares = [shamirs.shares(chunk, quantity=quantity, threshold=threshold) for chunk in chunks]
    combined_shares = combine_shares(shares)
    return combined_shares

# Function to recover the original secret from shares
def shamir_combine(shares, chunk_size):
    chunks = [shamirs.interpolate(share_set) for share_set in zip(*shares)]
    binary_str = ''.join(bin(chunk)[2:].zfill(chunk_size) for chunk in chunks)
    return int(binary_str, 2)

# XOR-based Secret Sharing using bytes
def encode(secret):
    M = bin(secret)[2:]
    if len(M) % 2 != 0:
        M += '0'
    half_len = len(M) // 2
    M1 = M[:half_len].encode()
    M2 = M[half_len:].encode()
    
    R1 = random.randbytes(len(M1))
    R2 = random.randbytes(len(M2))
    
    E1 = R1 + bytes(a ^ b for a, b in zip(M2, R2))
    E2 = bytes(a ^ b for a, b in zip(M1, R1)) + R2
    E3 = bytes(a ^ b for a, b in zip(M1, R2)) + bytes(a ^ b for a, b in zip(M2, R1))
    
    return E1, E2, E3

def decode(part1, part2, E1, E2, E3):
    half_len = len(part1) // 2

    if part1 == E1 and part2 == E2:
        R1 = part1[:half_len]
        R2 = part2[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R1))
        M2 = bytes(a ^ b for a, b in zip(part1[half_len:], R2))
    elif part1 == E1 and part2 == E3:
        R1 = part1[:half_len]
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
        R2 = bytes(a ^ b for a, b in zip(part1[half_len:], M2))
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
    elif part1 == E2 and part2 == E3:
        R2 = part1[half_len:]
        M1 = bytes(a ^ b for a, b in zip(part2[:half_len], R2))
        R1 = bytes(a ^ b for a, b in zip(part1[:half_len], M1))
        M2 = bytes(a ^ b for a, b in zip(part2[half_len:], R1))
    else:
        raise ValueError("Invalid parts provided for decoding.")

    M = M1 + M2
    return int.from_bytes(M, byteorder='big')

# List of input files
input_files = [
    'input_1_bytes.txt',
    'input_10_bytes.txt',
    'input_100_bytes.txt',
    'input_1000_bytes.txt',
    'input_10000_bytes.txt'
]

# Arrays to store average runtimes across files
avg_encoder_runtimes_shamir = []
avg_decoder_runtimes_shamir = []
avg_encoder_runtimes_xor = []
avg_decoder_runtimes_E1_E2 = []
avg_decoder_runtimes_E1_E3 = []
avg_decoder_runtimes_E2_E3 = []

n = 3  # Number of shares
k = 2  # Threshold

# Choose a chunk size that is smaller than the modulus (e.g., 32 bits)
chunk_size = 32

for input_file in input_files:
    # Read the secret from the file
    with open(input_file, 'rb') as f:
        secret = f.read().strip()

    # Convert the secret to an integer
    secret_int = int.from_bytes(secret, byteorder='big')

    # Arrays to store runtimes for this file
    encoder_runtimes_shamir = []
    decoder_runtimes_shamir = []
    encoder_runtimes_xor = []
    decoder_runtimes_E1_E2 = []
    decoder_runtimes_E1_E3 = []
    decoder_runtimes_E2_E3 = []

    # Perform the encoding and decoding multiple times (iterations) for averaging
    iterations = 10
    for _ in range(iterations):
        # Shamir's Scheme
        start_time = time.time()
        shamir_shares = shamir_split(secret_int, chunk_size, quantity=n, threshold=k)
        encoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        recovered_secret_int = shamir_combine(shamir_shares, chunk_size)
        decoder_runtimes_shamir.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        # XOR-based Secret Sharing using bytes
        start_time = time.time()
        E1, E2, E3 = encode(secret_int)
        encoder_runtimes_xor.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E1, E2, E1, E2, E3)
        decoder_runtimes_E1_E2.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E1, E3, E1, E2, E3)
        decoder_runtimes_E1_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

        start_time = time.time()
        decode(E2, E3, E1, E2, E3)
        decoder_runtimes_E2_E3.append((time.time() - start_time) * 1000000)  # Convert to microseconds

    # Calculating the averages for this file
    avg_encoder_runtimes_shamir.append(np.mean(encoder_runtimes_shamir))
    avg_decoder_runtimes_shamir.append(np.mean(decoder_runtimes_shamir))
    avg_encoder_runtimes_xor.append(np.mean(encoder_runtimes_xor))
    avg_decoder_runtimes_E1_E2.append(np.mean(decoder_runtimes_E1_E2))
    avg_decoder_runtimes_E1_E3.append(np.mean(decoder_runtimes_E1_E3))
    avg_decoder_runtimes_E2_E3.append(np.mean(decoder_runtimes_E2_E3))

# Print the calculated data
print("Average Encoder Runtimes (Shamir):", avg_encoder_runtimes_shamir)
print("Average Decoder Runtimes (Shamir):", avg_decoder_runtimes_shamir)
print("Average Encoder Runtimes (XOR):", avg_encoder_runtimes_xor)
print("Average Decoder Runtimes (E1,E2):", avg_decoder_runtimes_E1_E2)
print("Average Decoder Runtimes (E1,E3):", avg_decoder_runtimes_E1_E3)
print("Average Decoder Runtimes (E2,E3):", avg_decoder_runtimes_E2_E3)

# Plotting the results as line graphs
plt.figure(figsize=(14, 7), dpi=100)

plt.subplot(1, 2, 1)
plt.plot(input_files, avg_encoder_runtimes_shamir, label='Shamir Encoder', marker='o', color='#1f77b4')
plt.plot(input_files, avg_encoder_runtimes_xor, label='XOR Encoder', marker='o', color='#ff7f0e')
plt.title('Average Encoding Runtime for Different Inputs', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(input_files, avg_decoder_runtimes_shamir, label='Shamir Decoder', marker='o', color='#1f77b4')
plt.plot(input_files, avg_decoder_runtimes_E1_E2, label='E1,E2 Decoder', marker='o', color='#ff7f0e')
plt.plot(input_files, avg_decoder_runtimes_E1_E3, label='E1,E3 Decoder', marker='o', color='#2ca02c')
plt.plot(input_files, avg_decoder_runtimes_E2_E3, label='E2,E3 Decoder', marker='o', color='#d62728')
plt.title('Average Decoding Runtime for Different Inputs', fontsize=24)
plt.ylabel('Time (microseconds)', fontsize=24)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Sample data for the line graph
input_sizes = ['1 byte', '10 bytes', '100 bytes', '1 KB', '10 KB']
encoder_runtimes_shamir = [96.94099426269531, 34.57069396972656, 194.09656524658203, 1652.693748474121, 185192.03662872314]
encoder_runtimes_xor = [5.53131103515625, 14.662742614746094, 120.47290802001953, 881.9580078125, 8423.900604248047]

# Plotting the line graph
plt.figure(figsize=(12, 8))

plt.plot(input_sizes, encoder_runtimes_shamir, marker='o', linestyle='-', color='blue', label='Shamir')
plt.plot(input_sizes, encoder_runtimes_xor, marker='o', linestyle='-', color='green', label='XOR')

# Adding labels and title
plt.xlabel('Input Size', fontsize=20)
plt.ylabel('Time (microseconds)', fontsize=20)
plt.title('Encoder Runtimes for Various Input Sizes', fontsize=24)

# Adding legend
plt.legend(fontsize=16)

# Adjusting the tick label size
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

# Setting a logarithmic scale for the y-axis
plt.yscale('log')

# Display the plot
plt.show()


In [ ]:
# Plotting the line graph with enhanced aesthetics
plt.figure(figsize=(12, 8))

# Plotting the Shamir line with enhancements
plt.plot(input_sizes, encoder_runtimes_shamir, marker='o', linestyle='-', color='blue', 
         label='Shamir', markersize=10, linewidth=3)

# Plotting the XOR line with enhancements
plt.plot(input_sizes, encoder_runtimes_xor, marker='s', linestyle='--', color='green', 
         label='XOR', markersize=10, linewidth=3)

# Adding labels and title with enhanced font styles
plt.xlabel('Input Size', fontsize=22, fontweight='bold')
plt.ylabel('Time (microseconds)', fontsize=22, fontweight='bold')
plt.title('Encoder Runtimes for Various Input Sizes', fontsize=26, fontweight='bold')

# Adding grid for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Adding legend with enhanced font size
plt.legend(fontsize=18, loc='upper left')

# Adjusting the tick label size and style
plt.xticks(fontsize=18, fontweight='bold')
plt.yticks(fontsize=18, fontweight='bold')

# Setting a logarithmic scale for the y-axis
plt.yscale('log')

# Adding a custom background color
plt.gca().set_facecolor('#f0f0f0')

# Display the plot
plt.show()


In [ ]:
# Sample data for the decoding time line graph
input_sizes = ['1 byte', '10 bytes', '100 bytes', '1 KB', '10 KB']
decoder_runtimes_shamir = [81.634521484375, 243.25847625732422, 2064.53800201416, 17870.450019836426, 168733.14380645752]
decoder_runtimes_e1_e2 = [2.7179718017578125, 6.4849853515625, 40.0543212890625, 385.14137268066406, 3749.227523803711]
decoder_runtimes_e1_e3 = [3.1948089599609375, 8.702278137207031, 60.820579528808594, 576.8060684204102, 5680.537223815918]
decoder_runtimes_e2_e3 = [2.956390380859375, 8.749961853027344, 65.58895111083984, 579.0948867797852, 5722.332000732422]

# Plotting the line graph with data labels for decoding times

plt.figure(figsize=(12, 8))

# Plotting the Shamir line
plt.plot(input_sizes, decoder_runtimes_shamir, marker='o', linestyle='-', color='navy', 
         label='Shamir', markersize=8, linewidth=2)

# Plotting the E1,E2 line
plt.plot(input_sizes, decoder_runtimes_e1_e2, marker='s', linestyle='--', color='darkgreen', 
         label='E1,E2', markersize=8, linewidth=2)

# Plotting the E1,E3 line
plt.plot(input_sizes, decoder_runtimes_e1_e3, marker='^', linestyle=':', color='darkred', 
         label='E1,E3', markersize=8, linewidth=2)

# Plotting the E2,E3 line
plt.plot(input_sizes, decoder_runtimes_e2_e3, marker='d', linestyle='-.', color='purple', 
         label='E2,E3', markersize=8, linewidth=2)

# Adding labels and title with professional font styles
plt.xlabel('Input Size', fontsize=22)
plt.ylabel('Time (microseconds)', fontsize=22)
plt.title('Decoder Runtimes for Various Input Sizes', fontsize=24)

# Adding data labels to the points
for i, txt in enumerate(decoder_runtimes_shamir):
    plt.text(input_sizes[i], txt, f'{txt:.2f}', fontsize=14, ha='right', va='bottom')

for i, txt in enumerate(decoder_runtimes_e1_e2):
    plt.text(input_sizes[i], txt, f'{txt:.2f}', fontsize=14, ha='left', va='bottom')

for i, txt in enumerate(decoder_runtimes_e1_e3):
    plt.text(input_sizes[i], txt, f'{txt:.2f}', fontsize=14, ha='left', va='top')

for i, txt in enumerate(decoder_runtimes_e2_e3):
    plt.text(input_sizes[i], txt, f'{txt:.2f}', fontsize=14, ha='right', va='top')

# Adding a grid for clarity, but subtle
plt.grid(True, which='both', linestyle='-', linewidth=0.7, alpha=0.7)

# Adding legend with appropriate font size
plt.legend(fontsize=16)

# Adjusting the tick label size and style
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

# Setting a logarithmic scale for the y-axis
plt.yscale('log')

# Display the plot
plt.show()
